In [0]:
%sql
select * from  details;

col_name,data_type,comment
id,bigint,null
insurer_id,bigint,null
relationship,string,null


In [0]:
%sql
create table temp as  (select h.id as source_id,
h.insurer_id as subscriber_id,
d.first_name,
d.middle_name,
d.last_name,
d.email,
h.relationship,
d.ssn,
d.gender,
d.religion,
d.marital_status,
d.date_of_birth,
d.deceased_date,
d.spoken_language_1,
d.spoken_language_2
,d.company,
d.job_role,
d.job_hiredate,
c.phone,
c.usage_type,
a.city,
a.issued_date,
a.address_line_1,
a.address_line_2,
a.state,
a.address_type,
a.zipcode
from 
header h 
left join details d on h.id=d.id 
left join contactinfo c on h.id=c.id 
left join address a on h.id=a.id)


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from temp limit 5;

source_id,subscriber_id,first_name,middle_name,last_name,email,relationship,ssn,gender,religion,marital_status,date_of_birth,deceased_date,spoken_language_1,spoken_language_2,company,job_role,job_hiredate,phone,usage_type,city,issued_date,address_line_1,address_line_2,state,address_type,zipcode
70001,40184,Hettie,null,Keenlayside,jkeenlayside0@disqus.com,child,168-92-1075,F,Buddhism,Widowed,8/5/1939,null,West Frisian,Swahili,Gabcube,Clinical Specialist,1/29/1964,(455) 3130004,Work,New Kyle,5/26/16,4307 Ashley Village Suite 758,null,North Dakota,Mail,87337
70001,40184,Hettie,null,Keenlayside,jkeenlayside0@disqus.com,child,168-92-1075,F,Buddhism,Widowed,8/5/1939,null,West Frisian,Swahili,Gabcube,Clinical Specialist,1/29/1964,(455) 3130004,Work,Griffinhaven,6/10/07,046 Cox Lights,Suite 600,South Carolina,Residental,71709
70002,40092,Reade,null,Laverenz,dlaverenz1@senate.gov,friend,782-24-9907,M,Christianity,Widowed,5/14/1941,null,Swati,Danish,Skibox,Staff Scientist,5/18/1958,(994) 4561640,Work,North Heather,12/1/22,737 Banks Row,Apt. 505,Arkansas,Residental,2341
70002,40092,Reade,null,Laverenz,dlaverenz1@senate.gov,friend,782-24-9907,M,Christianity,Widowed,5/14/1941,null,Swati,Danish,Skibox,Staff Scientist,5/18/1958,(994) 4561640,Work,South Natalie,9/15/19,183 Dalton Viaduct,Suite 844,Oregon,Mail,77714
70003,40233,Minnnie,null,Baack,dbaack2@sina.com.cn,spouse,726-01-1271,F,Buddhism,Married,11/20/1982,null,Swati,null,Dabjam,Paralegal,6/10/2011,(771) 6498755,Work,Churchbury,3/9/07,807 Jesus Mills Suite 598,Suite 735,Texas,Mail,97223


In [0]:
%sql
create table target as (
select 
source_id,
subscriber_id,
first_name,
middle_name,
last_name,
CASE
  WHEN gender = 'M' THEN 'Mr.'
  WHEN gender = 'F' AND marital_status in ('Widowed','Married') THEN 'Mrs'
  WHEN gender = 'F' AND marital_status = 'Unmarried' THEN 'Miss'
END as prefix,
CASE
      WHEN first_name = middle_name AND relationship = 'child' THEN 'Jr.'
        WHEN job_role LIKE '%Engineer%' THEN 'Er'
        WHEN job_role LIKE '%Nurse%' THEN 'RN'
        WHEN job_role LIKE '%Analyst%' THEN 'Analyst'
        WHEN job_role LIKE '%Manager%' THEN 'Mgr'
        WHEN job_role LIKE '%Administrator%' THEN 'Admin'
        WHEN job_role LIKE '%Developer%' THEN 'Dev'
        WHEN job_role LIKE '%Assistant%' THEN 'Asst'
        WHEN job_role LIKE '%Technician%' THEN 'Tech'
        WHEN job_role LIKE '%Account%' THEN 'Acct'
        WHEN job_role LIKE '%Biostatistician%' THEN 'BioStat'
        WHEN job_role LIKE '%Health Coach%' THEN 'HlthCoach'
        WHEN job_role LIKE '%Designer%' THEN 'Designer'
        WHEN job_role LIKE '%Statistician%' THEN 'Stat'
        WHEN job_role LIKE '%Programmer%' THEN 'Prog'
        WHEN job_role LIKE '%Coordinator%' THEN 'Coord'
        WHEN job_role LIKE '%Automation Specialist%' THEN 'AutoSpec'
        WHEN job_role LIKE '%VP%' THEN 'VP'
        WHEN job_role LIKE '%Geologist%' THEN 'Geol'
      ELSE null
    end as suffix_name,
concat(ifnull(first_name,"" )," ",ifnull(middle_name,"")," ",ifnull(last_name,"")) as name,
'Nova Healthcare' as record_source,
current_timestamp() as record_created_ts,
"True" as is_verified,

array_agg(distinct(usage_type,phone)) as phones,
CASE
     WHEN email LIKE '%@%.__%' THEN email
     ELSE NULL
     END as email,array_agg(struct((address_type,address_line_1,address_line_2,city,state,
case when len(zipcode) > 5 then left(zipcode,5)
     when len(zipcode) = 5 then zipcode end,
case when len(zipcode) > 5 then right(zipcode,4)
     when len(zipcode) = 4 then zipcode end))) as addresses,
False as privacy_preference,
ssn as national_id,
gender,
marital_status,
to_date(date_of_birth,'m/d/yyyy') as date_of_birth,
year(to_date(date_of_birth,'m/d/yyyy')) as year_of_birth,
CASE
    WHEN deceased_date is null then 'No' else 'Yes'
END as deceased_ind,
CASE WHEN deceased_date is not null THEN timestampdiff(year,to_date(deceased_date,'m/d/yyyy'),to_date(date_of_birth,'m/d/yyyy')) 
END as deceased_age,
CASE
      WHEN spoken_language_1 IS NOT null AND spoken_language_2 IS NOT null THEN ARRAY(spoken_language_1,spoken_language_2)
      WHEN spoken_language_1 IS null AND spoken_language_2 IS NOT null THEN ARRAY(spoken_language_2)
      WHEN spoken_language_1 IS NOT null AND spoken_language_2 IS null THEN ARRAY(spoken_language_1)
      WHEN spoken_language_1 == spoken_language_2 THEN ARRAY(spoken_language_1)
      ELSE null
END AS languages,
array(struct(company as employer_name,
                  job_role as employee_role,
                  case when deceased_date is not null then 'Inactive' else 'Active' end as employee_status,
                  case when job_hiredate > date_of_birth then job_hiredate else null end as employee_hiredate)) as employment,
map('relationship :',relationship,'religion :',religion) as additional_source_value

from temp
group by source_id,
subscriber_id,
first_name,
middle_name,
last_name,
email,
relationship,
ssn,
gender,
religion,
marital_status,
date_of_birth,
deceased_date,
spoken_language_1,
spoken_language_2,
company,
job_role,
job_hiredate,
phone,
usage_type,
city,
issued_date,
address_line_1,
address_line_2,
state,
address_type,
zipcode
)




num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from target limit 5;

source_id,subscriber_id,first_name,middle_name,last_name,prefix,suffix_name,name,record_source,record_created_ts,is_verified,phones,email,addresses,privacy_preference,national_id,gender,marital_status,date_of_birth,year_of_birth,deceased_ind,deceased_age,languages,employment,additional_source_value
70001,40184,Hettie,null,Keenlayside,Mrs,null,Hettie Keenlayside,Nova Healthcare,2024-02-02T14:18:48.921+0000,True,"List(List(Work, (455) 3130004))",jkeenlayside0@disqus.com,"List(List(List(Residental, 046 Cox Lights, Suite 600, Griffinhaven, South Carolina, 71709, null)))",false,168-92-1075,F,Widowed,1939-01-05,1939,No,null,"List(West Frisian, Swahili)","List(List(Gabcube, Clinical Specialist, Active, null))","Map(relationship : -> child, religion : -> Buddhism)"
70001,40184,Hettie,null,Keenlayside,Mrs,null,Hettie Keenlayside,Nova Healthcare,2024-02-02T14:18:48.921+0000,True,"List(List(Work, (455) 3130004))",jkeenlayside0@disqus.com,"List(List(List(Mail, 4307 Ashley Village Suite 758, null, New Kyle, North Dakota, 87337, null)))",false,168-92-1075,F,Widowed,1939-01-05,1939,No,null,"List(West Frisian, Swahili)","List(List(Gabcube, Clinical Specialist, Active, null))","Map(relationship : -> child, religion : -> Buddhism)"
70002,40092,Reade,null,Laverenz,Mr.,null,Reade Laverenz,Nova Healthcare,2024-02-02T14:18:48.921+0000,True,"List(List(Work, (994) 4561640))",dlaverenz1@senate.gov,"List(List(List(Residental, 737 Banks Row, Apt. 505, North Heather, Arkansas, null, 2341)))",false,782-24-9907,M,Widowed,1941-01-14,1941,No,null,"List(Swati, Danish)","List(List(Skibox, Staff Scientist, Active, 5/18/1958))","Map(relationship : -> friend, religion : -> Christianity)"
70002,40092,Reade,null,Laverenz,Mr.,null,Reade Laverenz,Nova Healthcare,2024-02-02T14:18:48.921+0000,True,"List(List(Work, (994) 4561640))",dlaverenz1@senate.gov,"List(List(List(Mail, 183 Dalton Viaduct, Suite 844, South Natalie, Oregon, 77714, null)))",false,782-24-9907,M,Widowed,1941-01-14,1941,No,null,"List(Swati, Danish)","List(List(Skibox, Staff Scientist, Active, 5/18/1958))","Map(relationship : -> friend, religion : -> Christianity)"
70003,40233,Minnnie,null,Baack,Mrs,null,Minnnie Baack,Nova Healthcare,2024-02-02T14:18:48.921+0000,True,"List(List(Work, (771) 6498755))",dbaack2@sina.com.cn,"List(List(List(Mail, 807 Jesus Mills Suite 598, Suite 735, Churchbury, Texas, 97223, null)))",false,726-01-1271,F,Married,1982-01-20,1982,No,null,List(Swati),"List(List(Dabjam, Paralegal, Active, 6/10/2011))","Map(relationship : -> spouse, religion : -> Buddhism)"


In [0]:
%sql
z